In [1]:
import os
import pandas as pd

df = pd.read_csv("fin_bert2000.csv",error_bad_lines=False)
df = df.drop(columns=['company', 'year','pena_n'])
df = df.dropna(axis = 0,how = 'any')

C:\Users\DC\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df

,text_a,text_b,label
0,前一年度營業結果營業實施成果本公司於104年10月15日與中國信託金融控股股份有限公司(以下...,並提供保險顧問團隊輔訓服務深耕夥伴關係。直效行銷:持續增加電銷人員席次並提升產能。保險經紀人...,Y
1,"﻿前一年度營業結果105年度營業計畫實施成果105年,受新興市場成長動能放緩、美日歐表現不如...","強化財務結構降低營運及財務風險,提升營運績效1優化資產配置,佈局於收益率較佳之資產,提高經常...",Y
2,"﻿105年度營業結果回顧105年,受惠於國際電子產品需求增溫、原油及原物料價格止跌回穩,出口...","632件,成長率04%。保險給付為1,099億4,849萬元,較去年度增加88億1,053萬...",Y
3,"特別為樂退經濟需求,量身訂作「展鑫人生利率變動型終身還本保險」,自選65歲三種樂退年齡,提供...","並藉由完整的選、用、育、留增員架構,全面提高增員動能及業務員定著率,藉以提升正式組長數；並積...",Y
4,﻿前一年度營業結果105年度營業計畫實施成果本公司105年度經營績效報告如下：新契約保費收入...,"以持續開拓利基商品及提供完整客戶服務。通路策略深耕外部通路並強化類策盟合作關係,鞏固市場地位...",Y
...,...,...,...
175,"壹‧董事長劉炳輝先生106年度營業報告經濟局勢回顧106年,幾乎所有經濟調研機構在經濟報告上...","本行於106年度通過核准增設2處分行,分別設置於金融服務欠缺地區(高雄市燕巢區、苗栗縣銅鑼鄉...",N
176,"各位股東女士、先生,您好:在信用評等方面,惠譽信評於民國106年授予本行之評等結果為:國內長...","讓客戶享受金融科技帶來的便利。在企業金融業務方面,本行精準掌握產業脈動及市場需求,發揮集團遍...",Y
177,"106年以來,全球經濟成長動能增強,先進、新興與開發中經濟體同步復甦,國際主要經濟預測機構多...","且政府積極推動前瞻基礎建設及振興經濟六大措施,包括公務員加薪、推動稅改、加速投資臺灣、法規鬆...",N
178,"前言民國106年,全球經濟同步復甦,景氣穩健擴張,本行兼顧資產安全、資金充裕與資本充足,穩健...","105億元。重要之經營政策基本政策:秉持誠信穩健經營,健康均衡永續發展,創造良好經營績效。業...",Y


In [4]:
df_train = df.head(10)
df_test = df.tail(3)

df_train.to_csv("train.tsv", sep="\t", index=False)
print("訓練樣本數：", len(df_train))

df_test.to_csv("test.tsv", sep="\t", index=False)
print("預測樣本數：", len(df_test))

訓練樣本數： 10
預測樣本數： 3


In [5]:
df.label.value_counts() / len(df)

N    0.55618
Y    0.44382
Name: label, dtype: float64

In [6]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
import torch
from transformers import BertTokenizer,AutoTokenizer
from IPython.display import clear_output
    
class FakeNewsDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'Y': 0, 'N': 1}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 第二個句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞

#PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型
#tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained("schen/longformer-chinese-base-4096")
trainset = FakeNewsDataset("train", tokenizer=tokenizer)

In [9]:
for i in iter(trainset):
    data = i
    tokens_tensor, segments_tensor, label_tensor = data
    print(tokens_tensor.shape)

torch.Size([1779])
torch.Size([1502])
torch.Size([1921])
torch.Size([2366])
torch.Size([1691])
torch.Size([2482])
torch.Size([2525])
torch.Size([2101])
torch.Size([1889])
torch.Size([1299])


In [10]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `FakeNewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 4
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [11]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
text_a, text_b, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text_a}
句子 2：{text_b}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子 1：前一年度營業結果營業實施成果本公司於104年10月15日與中國信託金融控股股份有限公司(以下簡稱中信金控)進行股份轉換,並成為中信金控持股百分之百之子公司。中信金控自100年11月接手大都會國際人壽(該公司於101年1月更名為中國信託人壽保險股份有限公司,以下簡稱中信人壽),中信人壽又於103年併購宏利人壽台灣分公司。為有效整合資源,以提升綜效,105年1月1日中信金控旗下兩家壽險子公司中信人壽與台灣人壽合併,資產已破兆,105年整體營運狀況良好,茲將經營績效報告如下:初年度保費為1,399億元,業界排名第4名,銀行保險、保險經紀人及代理人、直效行銷通路皆排名業界前三名,且業務部隊更加擴大,擁有超過九千名業務員。總保費為2,524億,業界排名第5名。可運用資金12,051億元。上年度預算執行情形:合併元年因進行整併,公司營運得以發揮綜效,預算執行情形良好,合併稅後淨利為49億元。財務收支及獲利能力分析合併稅後淨利‥34億元。合併資產總額:12,856億元。基本每股盈餘‥61元。研究發展狀況商品面傳統型商品:即時滿足通路及市場需求,搶佔既有市場,並因應市場及法令變化,快速推出商品,搶得先機。如:因應實物給付保單開放,105年11月為首家推出連結殯葬服務實物給付保單之壽險業者。投資型商品:除持續開發多檔臺外幣投資型商品及投資型商品內扣式附約保障平台,滿足消費者投資與保障的需求之外,亦積極開發附保證給付商品及連結新型態標的之投資型商品,如:105年12月推出附保證給付之商品。服務面:持續強化客戶及通路服務,提升服務品質與便利性,如:新增繳費管道。本年度營業概要經營方針於商品、通路、投資、服務面皆持續精進,銷售能帶來長期合理收益之商品,持續擴大公司規模,為公司創造長期價值。通路:持續發展多元通路,並致力於引進具價值之契約,以提升公司價值,各通路營運計畫如下:銀行保險:於銀行通路推動傳統型期繳及投資型商品銷售
句子 2：並提供保險顧問團隊輔訓服務深耕夥伴關係。直效行銷:持續增加電銷人員席次並提升產能。保險經紀人及代理人:以核心保險經紀人及代理人營運模式增加通路黏著度。業務員:提升業務員產能並持續擴大業務員組織。企業保險:完善基礎建設,轉型為單一窗口多元產品線銷售與服務模式,推廣團體年金。商品:滿足客戶多元需求,並快速反應市場、通路需求及因應經濟環境及政

In [12]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([4, 2366]) 
tensor([[ 101, 1184,  671,  ...,    0,    0,    0],
        [ 101, 1184,  671,  ...,    0,    0,    0],
        [ 101, 8423, 2399,  ...,    0,    0,    0],
        [ 101, 4294, 1162,  ..., 6342, 2692,  102]])
------------------------
segments_tensors.shape = torch.Size([4, 2366])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1]])
------------------------
masks_tensors.shape    = torch.Size([4, 2366])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])
------------------------
label_ids.shape        = torch.Size([4])
tensor([0, 0, 0, 0])



In [23]:
for i in iter(trainloader):
    data = i
    tokens_tensors, segments_tensors,masks_tensors, label_ids = data
    
    print(tokens_tensors.shape)
    print(masks_tensors.shape)
    print(label_ids.shape)

torch.Size([4, 2366])
torch.Size([4, 2366])
torch.Size([4])
torch.Size([4, 2525])
torch.Size([4, 2525])
torch.Size([4])
torch.Size([2, 1889])
torch.Size([2, 1889])
torch.Size([2])


In [15]:
from transformers import BertForSequenceClassification,AutoModel

PRETRAINED_MODEL_NAME = "schen/longformer-chinese-base-4096"
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
#model = AutoModel.from_pretrained("schen/longformer-chinese-base-4096")
clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [16]:
model.config

BertConfig {
  "_name_or_path": "schen/longformer-chinese-base-4096",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

In [17]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cpu
classification acc: 0.7


In [ ]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")

In [ ]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 3  # 幸運數字
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))